# 关键词抽取

## 基于TF-IDF的关键词抽取

import jieba.analyse

* jieba.analyse.extract_tags(sentence, topK=20, withWeight=False, allowPOS=())
    * sentence 为待提取的文本
    * topK 为返回几个 TF/IDF 权重最大的关键词，默认值为 20
    * withWeight 为是否一并返回关键词权重值，默认值为 False
    * allowPOS 仅包括指定词性的词，默认值为空，即不筛选

In [1]:
import jieba.analyse as analyse
import pandas as pd
df = pd.read_csv("./origin_data/technology_news.csv", encoding='utf-8')
df = df.dropna()
lines=df.content.values.tolist()
content = "".join(lines)
keywords = analyse.extract_tags(content, topK=30, withWeight=False, allowPOS=())
print(keywords)

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\ADMINI~1\AppData\Local\Temp\jieba.cache
Loading model cost 0.738 seconds.
Prefix dict has been built successfully.


['用户', '2016', '互联网', '手机', '平台', '人工智能', '百度', '2017', '智能', '技术', '数据', '360', '服务', '直播', '产品', '企业', '安全', '视频', '移动', '应用', '网络', '行业', '游戏', '机器人', '电商', '内容', '中国', '领域', '通过', '发展']


In [2]:
import jieba.analyse as analyse
import pandas as pd
df = pd.read_csv("./origin_data/military_news.csv", encoding='utf-8')
df = df.dropna()
lines=df.content.values.tolist()
content = "".join(lines)
print("  ".join(analyse.extract_tags(content, topK=30, withWeight=False, allowPOS=())))

航母  训练  海军  中国  官兵  部队  编队  作战  10  任务  美国  导弹  能力  20  2016  军事  无人机  装备  进行  记者  我们  军队  安全  保障  12  战略  军人  日本  南海  战机


## 基于 TextRank 算法的关键词抽取

* jieba.analyse.textrank(sentence, topK=20, withWeight=False, allowPOS=('ns', 'n', 'vn', 'v')) 直接使用，接口相同，注意默认过滤词性。
* jieba.analyse.TextRank() 新建自定义 TextRank 实例

In [3]:
import jieba.analyse as analyse
import pandas as pd
df = pd.read_csv("./origin_data/military_news.csv", encoding='utf-8')
df = df.dropna()
lines=df.content.values.tolist()
content = "".join(lines)

print("  ".join(analyse.textrank(content, topK=20, withWeight=False, allowPOS=('ns', 'n', 'vn', 'v'))))
print("---------------------我是分割线----------------")
print("  ".join(analyse.textrank(content, topK=20, withWeight=False, allowPOS=('ns', 'n'))))

中国  海军  训练  美国  部队  进行  官兵  航母  作战  任务  能力  军事  发展  工作  国家  问题  建设  导弹  编队  记者
---------------------我是分割线----------------
中国  海军  美国  部队  官兵  航母  军事  国家  任务  能力  导弹  技术  问题  日本  军队  编队  装备  系统  记者  战略


## LDA主题模型

### 用LDA主题模型建模，看看这些新闻主要在说哪些topic

首先我们要把文本内容处理成固定的格式，一个包含句子的list，list中每个元素是分词后的词list。类似下面这个样子。

[[第，一，条，新闻，在，这里],[第，二，条，新闻，在，这里],[这，是，在，做， 什么],...]

In [5]:
from gensim import corpora, models, similarities
import gensim

### 载入停用词

In [6]:
stopwords=pd.read_csv("origin_data/stopwords.txt",index_col=False,quoting=3,sep="\t",names=['stopword'], encoding='utf-8')
stopwords=stopwords['stopword'].values

### 转换成合适的模式

In [7]:
import jieba
import pandas as pd
df = pd.read_csv("./origin_data/technology_news.csv", encoding='utf-8')
df = df.dropna()
lines=df.content.values.tolist()

sentences=[]
for line in lines:
    try:
        segs=jieba.lcut(line)
        segs = list(filter(lambda x:len(x)>1, segs))
        segs = list(filter(lambda x:x not in stopwords, segs))
        sentences.append(list(segs))
    except Exception as e:
        print(line)
        continue
# 处理完格式为[[第，一，条，新闻，在，这里],[第，二，条，新闻，在，这里],[这，是，在，做， 什么],...]
print(sentences[:2])

[['中新网', '日电', 'CES', '2017', '拉开', '大幕', '惊闻', 'AlphaGo', '升级版', 'Master', '迎来', '连胜', '人类', '顶尖', '围棋', '一周', '纷纷', '败给', '谷歌', '旗下', 'DeepMind', '团队', '打造', '围棋', '大脑', '聚焦', '人工智能', '本届', 'CES', '增添', '声势', '首次', '参展', '致力于', '打造', '原创', 'AI', '大脑', '中国', '深度', '学习', '领军', '企业', '商汤', '科技', '人工智能', '浪潮', '之巅', '众多', '业界', '前辈', '巨匠', '面前', '将会', '交出', '一份', '答卷'], ['徐立', '商汤', '科技', 'CEO', '谈起', '本次', '参展', '谈到', '成立', '刚刚', '两年', '创业', '公司', '参展', '展示', '最新', '人工智能', '技术', '产品', '表达', '人工智能', '理解', '人工智能', '特定', '领域', '超越', '人类', '广泛应用', '标志', 'Master', '胜利', '围棋', '世界', '开拓', '局面', '不谋而合']]


### 词袋模型

In [8]:
dictionary = corpora.Dictionary(sentences) # 根据现有数据生成词典
print(dictionary)
corpus = [dictionary.doc2bow(sentence) for sentence in sentences] # 对每句话，统计每个词语的频数，组成词袋模型

Dictionary(56565 unique tokens: ['2017', 'AI', 'AlphaGo', 'CES', 'DeepMind']...)


In [9]:
corpus[5] # 第5句话的词袋表示，意思为字典中index为21的词语出现了1次，index为25的词语出现了1次…………index为138的词语出现1次

[(21, 1),
 (39, 1),
 (61, 1),
 (68, 1),
 (78, 1),
 (82, 1),
 (91, 1),
 (92, 1),
 (103, 1),
 (104, 2),
 (105, 2),
 (124, 1),
 (129, 1),
 (130, 1),
 (131, 1),
 (132, 1),
 (133, 1),
 (134, 1),
 (135, 1),
 (136, 1),
 (137, 1),
 (138, 1)]

### LDA建模

In [10]:
# lda训练 corpus=转化为词袋的词语列表 id2word=词典 num_topics=主题数量
lda = gensim.models.ldamodel.LdaModel(corpus=corpus, id2word=dictionary, num_topics=20)

In [11]:
#我们查一下第3号分类，其中最常出现的单词是：
print(lda.print_topic(3, topn=5))

0.023*"品牌" + 0.023*"京东" + 0.016*"消费者" + 0.012*"摄像头" + 0.010*"线下"


In [12]:
#我们把所有的主题打印出来看看
for topic in lda.print_topics(num_topics=20, num_words=4):
    print(topic[1])

0.033*"数据" + 0.026*"技术" + 0.016*"人工智能" + 0.015*"发展"
0.042*"汽车" + 0.030*"驾驶" + 0.017*"PC" + 0.015*"防御"
0.064*"病毒" + 0.042*"联想" + 0.020*"识别" + 0.016*"电竞"
0.023*"品牌" + 0.023*"京东" + 0.016*"消费者" + 0.012*"摄像头"
0.011*"收入" + 0.010*"文件" + 0.009*"国家" + 0.009*"项目"
0.023*"机器人" + 0.014*"共享" + 0.012*"智能" + 0.012*"企业"
0.043*"百度" + 0.026*"服务" + 0.019*"业务" + 0.011*"提供"
0.076*"视频" + 0.058*"直播" + 0.031*"内容" + 0.029*"平台"
0.083*"手机" + 0.020*"360" + 0.014*"用户" + 0.009*"乐视"
0.050*"增长" + 0.040*"市场" + 0.032*"报告" + 0.031*"显示"
0.015*"高通" + 0.014*"金立" + 0.009*"政务" + 0.008*"手机"
0.018*"中国" + 0.018*"VR" + 0.017*"产品" + 0.011*"2017"
0.036*"用户" + 0.022*"营销" + 0.017*"百度" + 0.013*"商家"
0.034*"小米" + 0.021*"电信" + 0.017*"第一季度" + 0.015*"应急"
0.030*"用户" + 0.022*"攻击" + 0.019*"手机" + 0.018*"漏洞"
0.035*"互联网" + 0.021*"中国" + 0.021*"发展" + 0.016*"公司"
0.024*"联盟" + 0.022*"威胁" + 0.020*"国美" + 0.019*"IP"
0.033*"城市" + 0.020*"北京" + 0.020*"携程" + 0.017*"旅游"
0.031*"孩子" + 0.029*"家长" + 0.028*"永恒" + 0.020*"医疗"
0.100*"游戏" + 0.042*"勒索" + 0.019*"宽带" + 

In [13]:
#可以对新加入的文本，进行简单主题分类
text5 = ['徐立', '商汤', '科技', 'CEO', '谈起', '本次', '参展', '谈到', '成立', '刚刚', '两年', '创业', '公司', '参展', '展示', '最新', '人工智能', '技术', '产品', '表达', '人工智能', '理解', '人工智能', '特定', '领域', '超越', '人类', '广泛应用', '标志', 'Master', '胜利', '围棋', '世界', '开拓', '局面', '不谋而合']
bow = dictionary.doc2bow(text5)
ndarray = lda.inference([bow])[0]
print(''.join(text5))
for e, value in enumerate(ndarray[0]):
    print('主题%d推断值%.2f' % (e, value))

徐立商汤科技CEO谈起本次参展谈到成立刚刚两年创业公司参展展示最新人工智能技术产品表达人工智能理解人工智能特定领域超越人类广泛应用标志Master胜利围棋世界开拓局面不谋而合
主题0推断值28.87
主题1推断值0.05
主题2推断值1.26
主题3推断值0.05
主题4推断值0.05
主题5推断值1.37
主题6推断值0.05
主题7推断值0.05
主题8推断值0.05
主题9推断值0.05
主题10推断值0.05
主题11推断值0.05
主题12推断值0.05
主题13推断值0.05
主题14推断值0.98
主题15推断值0.05
主题16推断值0.05
主题17推断值2.70
主题18推断值0.05
主题19推断值0.05
